# Xtern Data Science Work Sample Assessment

## Data Sets
### Given Address's

In [ ]:
import pandas
import IPython

IPython.display.display(pandas.read_csv("address.csv"))

### Events In Indy
#### Information was found from both Downtown Indy and VisitIndy

In [ ]:
IPython.display.display(pandas.read_csv("eventsInIndy.csv"))

#### Information found from EventBrite.
* Idea on how to find this data was found here: [Stevesie Data](https://www.youtube.com/watch?v=GsLuV_Oc9h0). 
* Reason I had to go through this wierd way was due to the fact that EventBrite disabled their /search Endpoint. 
* See file [jsonToCSV](https://github.com/kcin999/Xtern-Projects/blob/main/jsonToCSV.py) in parent directory to see how I cleaned the data from [raw_eventbrite.json](https://github.com/kcin999/Xtern-Projects/blob/main/raw_eventbrite.json) and turned it into [eventbrite.csv](https://github.com/kcin999/Xtern-Projects/blob/main/eventbrite.csv)

In [ ]:
IPython.display.display(pandas.read_csv("eventbrite.csv"))